# Задание 1. 

In [131]:
import pandas as pd

import numpy as np

import matplotlib.pyplot as plt


%matplotlib inline

In [133]:
DATA_PATH = "C:\\Users\\DMITRY\\PY\\"

In [ ]:
def count_top3(years):
    columns = ['Name','Gender','Count']
    big_data = []
    for year in years: 
        big_data.append(pd.read_csv(
            (DATA_PATH + 'yob{0}.txt'.format(year)), 
            names=columns
        ))
    big_frame = pd.concat(big_data, ignore_index=True)
    big_frame = big_frame.groupby(['Name','Gender']).sum().sort_values(by='Count', ascending=False).reset_index()
    return big_frame.Name.tolist()[:3]

In [219]:
def count_dynamics(years):
    columns = ['Name','Gender','Count']
    output_dict = {}
    m_list = []
    f_list = []
    for year in years: 
        names = pd.read_csv(
            (DATA_PATH + 'yob{0}.txt'.format(year)), 
            names=columns
            )
        m_list.append(names[names.Gender=='M'].Count.sum())
        f_list.append(names[names.Gender=='F'].Count.sum())
    output_dict["M"] = m_list
    output_dict["F"] = f_list
    return output_dict

{'M': [150486, 1790871, 1962744], 'F': [299810, 1713259, 1814922]}
